<br>

# Introduction to Databases
## Overview

Invented by E.F. Codd. (1970's) a **relational database** stores and manages data using the relational model (set theory).

Data is organized into tables (relations) with **rows** and **columns**, and tables can be linked through **relationships**.

The goals of a DB are:

- Redundancy: Earlier systems often stored duplicate data in multiple places, making updates difficult and error-prone
- Inconsistency: Without a structured way to maintain relationships, data could become inconsistent across different parts of the system
- Independence: Users needed a way to work with data without worrying about how it was physically stored
- Integrity: Businesses needed reliable ways to ensure data accuracy and maintain relationships between related data

### DB's organize data into tables with known relationships, to:

1. Efficiently query and update related data
2. Enforce data integrity rules
3. Reduce data redundancy
4. Support concurrent users
5. Provide transaction safety

### A Relational Database Is...
- Organized collection of tables
- Collections of structured data
- Custom Views and Custom logic (functions)
- A large set of CLI tools

<BR>

### Why Relational Databases?
- Very high performance and reliability
- Data integrity
- Complex queries
- ACID compliance

<BR>

```
Other types of Databases
- Document (NoSQL)
- Key-value Store
- Graph
```


# Data Integrity

## ACID properties

The set of guarantees DB's make about how they handle operations to ensure that system remains safe, accurate and reliable, even when things go wrong.

### Atomicity

Transaction is all or nothing
No partial updates

### Consistency

Database moves from one valid state to another
All rules/constraints satisfied

### Isolation

Transactions execute as if they were sequential
Concurrent transactions don't interfere

### Durability

Committed changes are permanent
Survive system crashes

<br>

# Core Concepts
## Schemas

Schemas define a logical grouping of database objects:
- Blueprint of database structure
- Collection of tables, views, functions
- Defines organization and relationships

Generally speaking though it defines the structure of a table. You must define a schema and create the associated table before you may store or otherwise manipulate data.

### Composition of a table schema

Table schemas define all the properties (columns) that will be present in each row of our table.

The properties include:
- column name
- type (see below)
- size
- nullable
- default value
- auto-incrementing
- (and more)



## Data Types

Just like the type of a variable in Python, we must give each property in a schema a type.

> WARNING: In SQL, unlike Python, column types are immutable (columns don't change types):
- Text/String/Char
- Integer/Float (numeric)
- Dates
- Boolean
- Binary
- Timestamps
- And other specialized types
<BR>


## Tables

Tables are a collection of related data in rows and columns (think spreadsheet). They are the most basic storage unit in a database.

## Rows

Within a **table**, each `row` represents the collection of properties defined in the schema associated with the table

## Columns

Within a **row**, each `column` represents one property defined in the schema associated with the table

<BR>

### Let's create our first schema for use later:

In [1]:
import sqlite3  # a popular in-memory DB

# conn = sqlite3.connect(':memory:')  -- in MEMORY DB...
conn = sqlite3.connect('DSE200.db')  # USUAL WAY
cursor = conn.cursor()

def hasTable(conn, tableName):
  cursor = conn.cursor()
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (tableName,))
  return cursor.fetchone() is not None

# conn = sqlite3.connect(':memory:')  -- in MEMORY DB...
conn = sqlite3.connect('DSE200.db')  # USUAL WAY
cursor = conn.cursor()

# Check if the table exists
if not hasTable(conn, 'Users'):
  cursor.execute('CREATE TABLE IF NOT EXISTS Users (user_id INTEGER PRIMARY KEY AUTOINCREMENT, username VARCHAR(50) NOT NULL, email VARCHAR(100))')
  print("Table 'Users' created successfully.")
else:
  print("Table 'Users' already exists.")

conn.commit()
conn.close()


Table 'Usewtfrs' already exists.


# Constraints

Constraints are rules that enforce data integrity. The are conditions you specify about the data that the DB will enforce in order to catch potential problems and/or invalid relationships.
<br>


## **Primary Key** A unique ID that can't be NULL

```
CREATE TABLE Users (
    user_id INT PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100)
);
```

## **Foreign Key** - Reference to other table that ensure referential integrity


```
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    user_id INT,
    amount DECIMAL(10,2),
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);
```


## **UNIQUE** -- ensures unique value per row

```
CREATE TABLE Users (
    user_id INT PRIMARY KEY,
    username VARCHAR(50) UNIQUE,
    email VARCHAR(100) UNIQUE
);
```


## **CHECK** - Logical validation rule

```
CREATE TABLE Employees (
    id INT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    age INT CHECK (age >= 18),
    salary DECIMAL CHECK (salary > 0)
);
```

## **NOT NULL** -- ensure non-empty value
```
CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    price DECIMAL(10,2) NOT NULL,
    description TEXT
);
```

## **DEFAULT** - give fallback value if not otherwise provided
```
CREATE TABLE Posts (
    post_id INT PRIMARY KEY,
    title VARCHAR(200) NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    status VARCHAR(20) DEFAULT 'draft'
);
```

## Relationships

Another type of schema we can define in our database refers to the relationship between tables.

One-to-one relationships are the simplest form. Consider how each cellphone has one phone number, and each phone number is associated with one phone.

Another common type of relationship is one-to-many. Consider the messaging app on your phone. You have a list of contacts, and you may have conversations with any number of them. And in each conversation, there can be *many* messages. This is the nature of the relationships between tables.

The third and final kind of relationship is the many-to-many relationship. For example, each student can take multiple classes, and each class has multiple students.  (Many-to-many relationships require an intermeditate "pivot" table to make things work.)

<BR>

## SQL Commands (CLI)

Sending a command to a DB server is described as, "making a query". Examples include:

- SELECT: read data
- INSERT: add one or more rows
- UPDATE: change data in one or more rows
- DELETE: delete one or more rows



## Sample Tables

For our demonstration, we want to create some tables, and illustrated below:

### `Customers`
```
ID  | Name   | City
--------------------
1   | Alice  | NYC
2   | Bob    | LA
3   | Carol  | Miami
4   | Dan    | Boston
```
### `Orders`
```
OrderID | CustomerID | Product
----------------------------
101     | 1         | Book
102     | 1         | Pen
103     | 2         | Laptop
104     | 5         | Phone

```




In [2]:
from tabulate import tabulate
import sqlite3
conn = sqlite3.connect('DSE200a.db')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS Customers')
cursor.execute('DROP TABLE IF EXISTS Orders')

conn.commit()

# Cover method to display query results
# --------------------------------------------
def showRows(tableName, aCursor): # Added cursor argument
  headers = [description[0] for description in cursor.description]  # Get column names
  table_data = cursor.fetchall()
  print(f"{tableName} Data:")
  print(tabulate(table_data, headers=headers, tablefmt="grid"))  # Print using tabulate

# Cover method to show table structure
# --------------------------------------------
def describeTable(aCursor, tableName):
  aCursor.execute(f"PRAGMA table_info({tableName})")
  headers = [description[0] for description in aCursor.description]  # Get column names
  table_data = aCursor.fetchall()
  return tabulate(table_data, headers=headers, tablefmt="grid")+"\n"

# Cover method to determine if table exists...
# --------------------------------------------
def hasTable(conn, tableName):
    cursor = conn.cursor()
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{tableName}'")
    return cursor.fetchone() is not None

#-- Make the Customers table if it's not already made...

if not hasTable(conn, 'Customers'):
  cursor.execute('CREATE TABLE IF NOT EXISTS Customers (ID INTEGER PRIMARY KEY, Name TEXT, City TEXT)')
  print("\nTable 'Customers' created successfully.")
  print(describeTable(cursor, 'Customers'))
else:
  print("Table 'Customers' already exists.")

# Insert data into the Customers table
cursor.execute("INSERT INTO Customers (ID, Name, City) VALUES (1, 'Alice', 'NYC')")
cursor.execute("INSERT INTO Customers (ID, Name, City) VALUES (2, 'Bob', 'LA')")
cursor.execute("INSERT INTO Customers (ID, Name, City) VALUES (3, 'Carol', 'Miami')")
cursor.execute("INSERT INTO Customers (ID, Name, City) VALUES (4, 'Dan', 'Boston')")
conn.commit()

cursor.execute(f"SELECT * FROM Customers;") # Execute SELECT statement using the provided cursor
showRows("Customers", cursor)

#-- now do orders table...

if not hasTable(conn, 'Orders'):
  cursor.execute('CREATE TABLE IF NOT EXISTS Orders (OrderID INTEGER PRIMARY KEY, CustomerID INTEGER, Product TEXT, FOREIGN KEY (CustomerID) REFERENCES Customers(ID))')
  print("\nTable 'Orders' created successfully.")
  print(describeTable(cursor, 'Orders'))
else:
  print("Table 'Orders' already exists.")

# Insert data into the Orders table
cursor.execute("INSERT INTO Orders (OrderID, CustomerID, Product) VALUES (101, 1, 'Book')")
cursor.execute("INSERT INTO Orders (OrderID, CustomerID, Product) VALUES (102, 1, 'Pen')")
cursor.execute("INSERT INTO Orders (OrderID, CustomerID, Product) VALUES (103, 2, 'Laptop')")
cursor.execute("INSERT INTO Orders (OrderID, CustomerID, Product) VALUES (104, 5, 'Phone')")
conn.commit()

cursor.execute(f"SELECT * FROM Orders;") # Execute SELECT statement using the provided cursor
showRows("Orders", cursor)

# Close the cursor and connection
cursor.close()
conn.close();



Table 'Customers' created successfully.
+-------+--------+---------+-----------+--------------+------+
|   cid | name   | type    |   notnull | dflt_value   |   pk |
+=======+========+=========+===========+==============+======+
|     0 | ID     | INTEGER |         0 |              |    1 |
+-------+--------+---------+-----------+--------------+------+
|     1 | Name   | TEXT    |         0 |              |    0 |
+-------+--------+---------+-----------+--------------+------+
|     2 | City   | TEXT    |         0 |              |    0 |
+-------+--------+---------+-----------+--------------+------+

Customers Data:
+------+--------+--------+
|   ID | Name   | City   |
+======+========+========+
|    1 | Alice  | NYC    |
+------+--------+--------+
|    2 | Bob    | LA     |
+------+--------+--------+
|    3 | Carol  | Miami  |
+------+--------+--------+
|    4 | Dan    | Boston |
+------+--------+--------+

Table 'Orders' created successfully.
+-------+------------+---------+--------

## Kinds of JOINS

INNER JOIN
🔵 ⚪ = ⚫
A ∩ B

LEFT JOIN
🔵 ⚪ = 🔵
A + (A ∩ B)

RIGHT JOIN
🔵 ⚪ = ⚪
B + (A ∩ B)

FULL JOIN
🔵 ⚪ = 🔵 ⚪
A + B

## Inner Join
An INNER JOIN combines rows from two tables when there's a matching value in both tables.


In [3]:
# Execute the INNER JOIN

conn = sqlite3.connect('DSE200a.db')
cursor = conn.cursor()

# Execute the INNER JOIN query using cursor.execute()
cursor.execute("""
SELECT Customers.Name, Orders.Product
FROM Customers
INNER JOIN Orders ON Customers.ID = Orders.CustomerID
""")

# Now, instead of printing the results directly, use your 'showRows' function
showRows("Customers/Orders Inner Join", cursor)

# Close the cursor and connection
cursor.close()
conn.close();


Customers/Orders Inner Join Data:
+--------+-----------+
| Name   | Product   |
+========+===========+
| Alice  | Book      |
+--------+-----------+
| Alice  | Pen       |
+--------+-----------+
| Bob    | Laptop    |
+--------+-----------+


#### NOTE:
- Only matches shown
- Carol, Dan (no orders) excluded
- CustomerID 5 (no customer) excluded

<BR>

## LEFT JOIN

>  🔵 ⚪ = 🔵 &nbsp; &nbsp; &nbsp; A + (A ∩ B)

A LEFT JOIN keeps ALL records from the left (first) table and matches records from the right table where possible. If there's no match, it fills in NULL values.


In [4]:
# Execute the LEFT JOIN

conn = sqlite3.connect('DSE200a.db')
cursor = conn.cursor()

# Execute the INNER JOIN query using cursor.execute()
cursor.execute("""
SELECT Customers.Name, Orders.Product
FROM Customers
LEFT JOIN Orders ON Customers.ID = Orders.CustomerID
""")

# Now, instead of printing the results directly, use your 'showRows' function
showRows("Customers/Orders Inner Join", cursor)

# Close the cursor and connection
cursor.close()
conn.close();

Customers/Orders Inner Join Data:
+--------+-----------+
| Name   | Product   |
+========+===========+
| Alice  | Book      |
+--------+-----------+
| Alice  | Pen       |
+--------+-----------+
| Bob    | Laptop    |
+--------+-----------+
| Carol  |           |
+--------+-----------+
| Dan    |           |
+--------+-----------+


## RIGHT JOIN

> 🔵 ⚪ = ⚪ &nbsp; &nbsp; &nbsp; B + (A ∩ B)

A RIGHT JOIN is like a LEFT JOIN but reversed - it keeps ALL records from the right (second) table and matches from the left where possible. If there's no match, it fills in NULL values.

Note: SQLite doesn't support RIGHT JOIN, but here's an example to understand it:

```
SELECT Customers.Name, Orders.Product FROM Customers RIGHT JOIN Orders ON Customers.ID = Orders.CustomerID

+--------+-----------+-----------+
| ID     | Name      | City      |
+========+===========+===========+
| 1      | John      | NYC       |
+--------+-----------+-----------+
| 2      | Jane      | LA        |
+--------+-----------+-----------+
| 3      | Bob       | Chicago   |
+--------+-----------+-----------+
```


## FULL JOIN

> 🔵 ⚪ = 🔵 ⚪ &nbsp; &nbsp; &nbsp; A + B

A FULL JOIN combines a LEFT and RIGHT JOIN - it keeps ALL records from BOTH tables, even if they don't have matches. If there's no match on either side, it fills in NULL values. All records from both table appear.

Note: SQLite doesn't support FULL JOIN directly, but here's an example to understand it:

```
SELECT Customers.Name, Orders.Product
FROM Customers
FULL JOIN Orders ON Customers.ID = Orders.CustomerID
```

```
+-----------+-----------+
| Name      | Orders    |
+===========+===========+
| John      | Laptop    |
+-----------+-----------+
| John      | Phone     |
+-----------+-----------+
| Jane      | Tablet    |
+-----------+-----------+
| Bob       | NULL      |
+-----------+-----------+
| NULL      | Monitor   |
+-----------+-----------+
```




## Common JOIN problems
- Missing matches
- Multiple matches
- Performance issues
- Wrong JOIN type


## Indexes

To greatly improve performance, you can create any number "index" tables -- that automatically map specified keys with associated rows.
- Indexes take up extra space; don't overuse.  
- They update automatically

```
cursor.execute('CREATE INDEX idx_city ON Customers(City)')
```

<BR>



## Transactions

Transactions are groups of operations that are treated as a single unit of work. They follow the ACID properties:

- **Atomicity**: All operations in the transaction either complete successfully together, or none of them do

- **Consistency**: The database remains in a valid state before and after the transaction

- **Isolation**: Transactions don't interfere with each other when running concurrently

- **Durability**: Once a transaction is committed, the changes are permanent

### Simple Example
Transferring $100 from Account A to Account B requires two operations:

1. Deduct $100 from A
2. Add $100 to B

Without transactions, if the system crashes after step 1 but before step 2, money would simply disappear. With transactions, either both steps complete successfully, or neither does - maintaining data integrity.

<BR>
